In [1]:
import json
import sys


import numpy as np
import matplotlib.pyplot as plt

sys.path.append('../lccd_python/')
import blob_detector
import test_utils
import data_synthesizer
import oval_filter
from lccd import LCCD

In [10]:
import tifffile

arr = tifffile.imread('data/recording_20161007_143400.tif')

In [11]:
arr = arr[::6, :512, :512]
arr = np.transpose(arr, (1, 2, 0))

In [12]:
arr.shape

(512, 512, 225)

In [13]:
with open('../config/config.json', 'r') as f:
    config = json.load(f)
import pprint
pprint.pprint(config)

{'blob_detector': {'filtersize1': 100,
                   'filtersize2': 4,
                   'fsize': 7,
                   'max_area': 50,
                   'min_area': 20,
                   'sigma': 1.25,
                   'sparse': True},
 'lccd': {'frame_divider': 100},
 'roi_integration': {'max_area': 100,
                     'min_area': 20,
                     'overlap_threshold': 0.4,
                     'sparse': True}}


In [44]:
config["blob_detector"]["debug"]=True
config["roi_integration"]["sparse"] = True
lccd_sparse = LCCD(config)
config["blob_detector"]["sparse"] = False
config["roi_integration"]["sparse"] = False
lccd_dense = LCCD(config)
v = arr
v = (v - v.min()) / (v.max() - v.min())
tm1 = v[:, :, 0: 100]
tm2 = v[:, :, 100: 200]

In [21]:
roi_tm1_dense = lccd_dense.blob_detector.apply(tm1)
label_mat2 = roi_tm1_dense["label_mat2"]

In [38]:
%%time
roi_sparse = oval_filter.oval_filter(label_mat2, sparse=True)

CPU times: user 144 ms, sys: 0 ns, total: 144 ms
Wall time: 143 ms


In [39]:
%%time
roi_dense = oval_filter.oval_filter(label_mat2, sparse=False)

CPU times: user 589 ms, sys: 108 ms, total: 697 ms
Wall time: 698 ms


In [32]:
np.array_equal(roi_sparse.toarray(), roi_dense)

True

In [40]:
roi_b = lccd_dense.blob_detector.apply(tm2)["ROI"]
roi_a = roi_dense

In [41]:
%%time
res_dense = lccd_dense.roi_integration.apply(roi_a, roi_b)

CPU times: user 17min 8s, sys: 6min 32s, total: 23min 40s
Wall time: 1min 6s


In [45]:
%%time
res_sparse = lccd_sparse.roi_integration.apply(roi_a, roi_b)

CPU times: user 2.93 s, sys: 192 ms, total: 3.12 s
Wall time: 3.12 s


In [48]:
np.array_equal(res_sparse.toarray(), res_dense)

True